In [1]:
from kagglegym import make
import kagglegym
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import linear_model
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
env = make()
o = env.reset()
train = o.train

d_median= train.median(axis=0)
#train["nbnulls"]=train.isnull().sum(axis=1)

In [3]:
train = train.fillna(d_median)

In [4]:
params = {
    'booster': "gbtree", #gblinear? dart?,
    'silent': 1,
    'nthread': 8,
    'max_depth': 10,
    'eta':.1, 
    
    #'tree_method': 'exact'
    #'objective': 'reg:linear', # reg:tweedie?,
    
}
columns = ['technical_30', 'technical_20', 'technical_19', 'fundamental_11']

In [5]:
train_mat = xgb.DMatrix(train[columns], train.y.values)

In [6]:
# cv_xgb = xgb.cv(params = params, dtrain = train_mat, num_boost_round = 3000, nfold = 5,
#                 metrics = ['error'], # Make sure you enter metrics inside a list or you may encounter issues!
#                 early_stopping_rounds = 100, verbose_eval=20)

In [7]:
# cv_xgb

In [15]:
print type(train.y.values[:10])

<type 'numpy.ndarray'>


In [8]:
booster = xgb.train(params, train_mat, num_boost_round=10)

In [9]:
env = make()
o = env.reset()
print('Starting a new calculation for score')
rewards = []

print('Starting to fit a model')
while True:
    test_mat = xgb.DMatrix(o.features[columns].copy().fillna(d_median))
    prediction  = booster.predict(test_mat)
    target      = o.target
    target['y'] = prediction

    timestamp = o.features["timestamp"][0]
    if timestamp % 100 == 0:
        print(timestamp)

    o, reward, done, info = env.step(target)
    rewards.append(reward)
    if done: break

print info['public_score'], rewards

Starting a new calculation for score
Starting to fit a model
1000
1100
1200
1300
1400
1500
1600
1700
1800
-0.0359928757146 [-0.19638146677827431, -0.066877146510487942, -0.10979636105204485, -0.15765755832159317, -0.090773823921979163, -0.11973333854368196, -0.33615754264622871, -0.07872609404150209, -0.12886475268090727, -0.076415784499309386, -0.10124747139895725, -0.16978234277054155, 0.037001170196728529, 0.013100670950920273, -0.19552510051557306, -0.14111468874612595, -0.10513669747351866, -0.28304601367799642, -0.060033521537219692, -0.10662304337181659, -0.17540699633358156, 0.013115874744589218, -0.18982221187928652, -0.10784694133420687, -0.16352446105282056, -0.045113605844864686, -0.35295041078241152, -0.13435335520247726, -0.15860979190265609, -0.17632629795743857, -0.080212386735004512, -0.096260253740100249, -0.3126062968728478, -0.24787437400506729, -0.055068360140681546, 0.034066262847572443, -0.027927087357149279, -0.18834082622861439, -0.10175341716042857, -0.0703613